# Identifying Top Films Using Box Office Data

## Introduction
Video content is one of the most watched forms of entertainment today. As our company prepares to create a new movie studio, it is important to understand what kinds of films perform best. By looking into Box Office Data, this project aims to uncover key trends and insights that can support smarter decisions as we step into the film industry.

## Business Understanding
This project analyzes Box Office Data to uncover patterns in film performance. The goal is to support the company's entry into the film market by answering key business questions, such as:

 - What genres dominate the box office?

 - What revenue trends exist over time?

 - Which films and actors attract the highest audiences?

The insights will help the  head of our company’s new movie studio  to make informed decisions about what kinds of films to produce.

## Data Understanding
The data used in this project comes from several popular film industry sources, including:

  - `Box Office Mojo`: Revenue and performance data  
  - `IMDb`: Film titles, cast, and crew  
  - `Rotten Tomatoes`: Critic and audience ratings  
  - `TheMovieDB`: Genre and overview details  
  - `The Numbers`: Budgets and revenue

These datasets are stored in the `Data` folder. Since the files were collected from different sources, they vary in structure and formatting, hence I will be using pandas and SQLite3 to explore and combine these datasets.

## 1. Data Exploration
With the project clearly explained, i will now load and explore the datasets to be used for this analysis.

#### &nbsp;&nbsp;&nbsp;&nbsp; 1.1 Importing relevant libraries 

In [1]:
# Import relevant libraries
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#### &nbsp;&nbsp;&nbsp;&nbsp;1.2 Load the Sqlite dataset and query to get the relevant tables

In [10]:
# Establish a connection to database

conn = sqlite3.connect('Data/im.db')

# Display the tables From the database using pandas

pd.read_sql("""SELECT  *
            FROM sqlite_master ;
            """, conn)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


Query and preview the tables that are relevant that is `movie_basics` and `movie_ratings`.

In [35]:
# Query the movie_basic table
pd.read_sql("""SELECT * 
               FROM movie_basics
               LIMIT 10;
             """, conn)



,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
5,tt0111414,A Thin Life,A Thin Life,2018,75.0,Comedy
6,tt0112502,Bigfoot,Bigfoot,2017,NaN,"Horror,Thriller"
7,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy"
8,tt0139613,O Silêncio,O Silêncio,2012,NaN,"Documentary,History"
9,tt0144449,Nema aviona za Zagreb,Nema aviona za Zagreb,2012,82.0,Biography


In [34]:
# Query the movie_ratings table
pd.read_sql("""SELECT * 
                FROM movie_ratings
                LIMIT 10;
            """, conn)


,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
5,tt1069246,6.2,326
6,tt1094666,7.0,1613
7,tt1130982,6.4,571
8,tt1156528,7.2,265
9,tt1161457,4.2,148


Merge the two tables using the unique identifier `movie_id` present in both tables and call the new dataframe **`df_movies`**.

In [38]:
# Merge the two tables using Sql
df_movies = pd.read_sql("""SELECT *
               FROM movie_basics 
               JOIN movie_ratings 
               USING(movie_id);
            """, conn)

df_movies.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119


With the merged dataframe we can inspect the dataframe to check the structure, check null values and duplicates


In [39]:
# Check the summary of the dataframe
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         73856 non-null  object 
 1   primary_title    73856 non-null  object 
 2   original_title   73856 non-null  object 
 3   start_year       73856 non-null  int64  
 4   runtime_minutes  66236 non-null  float64
 5   genres           73052 non-null  object 
 6   averagerating    73856 non-null  float64
 7   numvotes         73856 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 4.5+ MB


The summary shows 73856 rows and 8 columns. The dataframe seems to have missing values.

In [50]:
# Check for duplicates
print(df_movies.duplicated().sum())

0


No duplicates recorded. Next we get an overview of the distributions  of the data.

In [56]:
#
df_movies.describe()

,start_year,runtime_minutes,averagerating,numvotes
count,73856.000000,66236.000000,73856.000000,7.385600e+04
mean,2014.276132,94.654040,6.332729,3.523662e+03
std,2.614807,208.574111,1.474978,3.029402e+04
min,2010.000000,3.000000,1.000000,5.000000e+00
25%,2012.000000,81.000000,5.500000,1.400000e+01
50%,2014.000000,91.000000,6.500000,4.900000e+01
75%,2016.000000,104.000000,7.400000,2.820000e+02
max,2019.000000,51420.000000,10.000000,1.841066e+06


In [49]:
# Check for missing values
print(df_movies.isnull().sum())


movie_id              0
primary_title         0
original_title        0
start_year            0
runtime_minutes    7620
genres              804
averagerating         0
numvotes              0
dtype: int64


Two columns runtime_minutes and genres have missing values which we will clean during the cleaning step.

#### &nbsp;&nbsp;&nbsp;&nbsp;1.2 Load the CSV dataset and explore

In [51]:
# Load the CSV file using pandas
df = pd.read_csv("Data/bom.movie_gross.csv.gz")

# Display the dataframe
df

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


We now inspect our dataframe to understand its structure by checking the summary, missing values and duplicates.

In [59]:
# Preview the  dataframe summary
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


The dataframe has 3387 rows and 5 columns. The dataframe clearly has missing values.

In [60]:
# Check for missing values
df.isnull().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

Three columns have missing values to be addressed during the cleaning part.

In [61]:
# Check for duplicates
df.duplicated().sum()

0